# EBAC - Regressão II - regressão múltipla

## Tarefa II

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings('ignore')


In [4]:
df = pd.read_csv('previsao_de_renda.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na base de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [7]:
# Carregando o dataset
df = pd.read_csv('previsao_de_renda.csv')

# Dividindo o dataset em conjuntos de treinamento e teste
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
# Removendo as colunas 'Unnamed: 0', 'data_ref' e 'id_cliente' dos conjuntos de treinamento e teste
train_df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)
test_df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1, inplace=True)

# Exibindo as informações atualizadas sobre os conjuntos de dados resultantes
train_df.info()



<class 'pandas.core.frame.DataFrame'>
Index: 11250 entries, 7410 to 7270
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   sexo                   11250 non-null  object 
 1   posse_de_veiculo       11250 non-null  bool   
 2   posse_de_imovel        11250 non-null  bool   
 3   qtd_filhos             11250 non-null  int64  
 4   tipo_renda             11250 non-null  object 
 5   educacao               11250 non-null  object 
 6   estado_civil           11250 non-null  object 
 7   tipo_residencia        11250 non-null  object 
 8   idade                  11250 non-null  int64  
 9   tempo_emprego          9313 non-null   float64
 10  qt_pessoas_residencia  11250 non-null  float64
 11  renda                  11250 non-null  float64
dtypes: bool(2), float64(3), int64(2), object(5)
memory usage: 988.8+ KB


In [8]:
# Preparando as variáveis categóricas e numéricas para a modelagem
categorical_cols = train_df.select_dtypes(include=['object', 'bool']).columns
numeric_cols = train_df.select_dtypes(include=['int64', 'float64']).columns.drop('renda')

# Criando o pré-processador com OneHotEncoder para categóricas e imputador para numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='median'), numeric_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ])

# Criando o modelo Ridge com diferentes valores de alpha
ridge_alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
ridge_models = {}
ridge_scores = {}

for alpha in ridge_alphas:
    # Criando o pipeline com o pré-processador e o modelo Ridge
    ridge_pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', Ridge(alpha=alpha))
    ])

    # Treinando o modelo no conjunto de treinamento
    ridge_pipe.fit(train_df, train_df['renda'])

    # Prevendo no conjunto de teste
    y_pred = ridge_pipe.predict(test_df)

    # Calculando o R² e armazenando o resultado
    r2 = r2_score(test_df['renda'], y_pred)
    ridge_models[alpha] = ridge_pipe
    ridge_scores[alpha] = r2

# Mostrando os resultados do R² para cada valor de alpha
ridge_scores


{0: 0.2690308242376488,
 0.001: 0.26903083448758025,
 0.005: 0.2690308754511588,
 0.01: 0.2690309265744735,
 0.05: 0.26903133234807475,
 0.1: 0.26903183169156664}

Os valores de R² são bastante similares, mas aumentam ligeiramente com o aumento do alpha. O melhor modelo, baseado em R², é o que utiliza alpha = 0.1, que apresenta o maior valor.

In [10]:
# Criando o modelo Lasso com diferentes valores de alpha
lasso_alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
lasso_models = {}
lasso_scores = {}

for alpha in lasso_alphas:
    # Criando o pipeline com o pré-processador e o modelo Lasso
    lasso_pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', Lasso(alpha=alpha, max_iter=10000))  # Aumentando o número de iterações para garantir convergência
    ])

    # Treinando o modelo no conjunto de treinamento
    lasso_pipe.fit(train_df, train_df['renda'])

    # Prevendo no conjunto de teste
    y_pred = lasso_pipe.predict(test_df)

    # Calculando o R² e armazenando o resultado
    r2 = r2_score(test_df['renda'], y_pred)
    lasso_models[alpha] = lasso_pipe
    lasso_scores[alpha] = r2

# Mostrando os resultados do R² para cada valor de alpha
lasso_scores


{0: 0.2690308242376489,
 0.001: 0.2690309164455068,
 0.005: 0.26903128175885405,
 0.01: 0.2690317304850405,
 0.05: 0.2690350036707603,
 0.1: 0.2690383035972087}

Até o momento, tanto a regressão Ridge quanto a regressão Lasso apresentaram resultados similares de R² em torno de 0.2690. A diferença entre eles, com base nos valores de alpha testados, foi mínima, com o melhor modelo de cada método mostrando melhorias muito pequenas ao aumentar o alpha.

In [12]:
# Criando um modelo de regressão linear para usar no RFE (Recursive Feature Elimination)
linear_regressor = LinearRegression()

# Criando o pipeline com o pré-processador e o RFE
stepwise_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('rfe', RFE(estimator=linear_regressor, n_features_to_select=1, step=1)),
    ('regressor', linear_regressor)
])

# Treinando o modelo no conjunto de treinamento
stepwise_model.fit(train_df, train_df['renda'])

# Prevendo no conjunto de teste
y_pred_stepwise = stepwise_model.predict(test_df)

# Calculando o R²
r2_stepwise = r2_score(test_df['renda'], y_pred_stepwise)
r2_stepwise


0.0856850765656113

Para escolher o melhor modelo entre Ridge, Lasso e Stepwise (regressão linear com RFE), consideramos tanto o desempenho (medido por R²) quanto as características de regularização e seleção de atributos:

    Ridge e Lasso: Ambos atingiram R² similares em torno de 0.2690, com a diferença que o Lasso pode zerar alguns coeficientes, fazendo uma seleção natural de variáveis.
    Stepwise: Apresentou um R² mais baixo de 0.0857, sugerindo que a seleção de atributos foi menos eficaz para este conjunto de dados.

Conclusão: O Lasso se destaca como a melhor opção. Ele não apenas alcança um bom R², mas também ajuda na interpretação do modelo ao identificar as variáveis mais impactantes, sendo uma escolha robusta para evitar o overfitting e simplificar o modelo.

In [14]:
# Recriando o pré-processador com transformações polinomiais e normalização
advanced_preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False))
        ]), numeric_cols),
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ])

# Recriando o modelo Lasso com transformações avançadas
advanced_lasso_model = Pipeline(steps=[
    ('preprocessor', advanced_preprocessor),
    ('regressor', Lasso(alpha=0.01, max_iter=10000))
])

# Treinando o modelo com as novas transformações no conjunto de treinamento
advanced_lasso_model.fit(train_df, train_df['renda'])

# Prevendo no conjunto de teste
y_advanced_pred = advanced_lasso_model.predict(test_df)

# Calculando o R² com as novas transformações
r2_advanced = r2_score(test_df['renda'], y_advanced_pred)
r2_advanced



0.270645014864743

In [15]:
# Criando um modelo de árvore de decisão com profundidade limitada para evitar overfitting
tree_model = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Reutilizando o pré-processador original sem transformações polinomiais
    ('regressor', DecisionTreeRegressor(max_depth=4))  # Usando uma árvore pequena com profundidade máxima de 4
])

# Treinando o modelo de árvore de decisão no conjunto de treinamento
tree_model.fit(train_df, train_df['renda'])

# Prevendo no conjunto de teste
y_tree_pred = tree_model.predict(test_df)

# Calculando o R² para a árvore de decisão
r2_tree = r2_score(test_df['renda'], y_tree_pred)
r2_tree


0.35795931804055003

Utilizando uma árvore de decisão com profundidade máxima de 4, conseguimos um R² de aproximadamente 0.358 na base de testes. Este resultado é superior ao que obtivemos com os modelos de regressão Lasso e Ridge, indicando que a árvore de decisão, mesmo sendo pequena, foi capaz de capturar melhor as relações entre as variáveis e a variável de resposta.

A árvore de decisão pode ser particularmente eficaz neste caso por sua habilidade de modelar relações não-lineares e interações entre variáveis sem a necessidade de especificação explícita, como é o caso das transformações polinomiais. 